In [ ]:
'''
    Quick Summary of What I have done here
    - Cleaned the data and identified columns with missing values
    - Used a custom seasonal_fill() function to smartly fill missing values by looking at the same calendar month across yearsü
    - Kept original columns intact, while creating _filled versions for transparency and comparison
    - Reorganized columns so originals sit next to their filled siblings
    - Verified column data types — all numeric, no hidden objects
    - Saved DataFrame as a CSV file for future use
    - Ready to launch dashboards with KPIs, filters, and trend visuals
'''
#For loading, manipulating, and analyzing tabular data
import pandas as pd
#For numerical operations, arrays, and mathematical functions
import numpy as np

In [ ]:
# --- Helper Functions ---

In [ ]:
# Combines multi-row headers into single clean column names by joining and cleaning text
def build_headers (header_rows):
    headers = header_rows.fillna('').astype(str).agg(' '.join, axis=0)
    headers = headers.str.strip().str.replace(r'\s+',' ', regex=True)
    return headers

In [ ]:
# Fills missing values using the average of same-month values from future or past entries
def seasonal_fill(series, month_col):
    #empty list to collect all filled values
    filled = [] 
    for idx in range(len(series)): ##loop over each row using its index
        value = series.iloc[idx] ##value is the actual data at that row (could be a number or NaN)
        month = month_col.iloc[idx] ##month grabs the calendar month for that row — helps us find other same-month values

        if pd.isna(value):
            # Try looking ahead first
            future_vals = series.iloc[idx+1:]
            future_same_month = future_vals[month_col.iloc[idx+1:] == month].dropna()

            if not future_same_month.empty:
                filled.append(future_same_month.mean())
            else:
                # Look back if no future values available
                past_vals = series.iloc[:idx]
                past_same_month = past_vals[month_col.iloc[:idx] == month].dropna()

                if not past_same_month.empty:
                    filled.append(past_same_month.mean())
                else:
                    filled.append(np.nan)  # still missing
        else:
            filled.append(value) #If the value isn’t missing, we simply keep it as-is
    return pd.Series(filled, index=series.index)


In [ ]:
# --- Main Workflow ---

In [ ]:
#URL of the raw Excel dataset to be imported
url='https://catalogue.data.gov.bc.ca/dataset/cace513c-9506-4f20-8dd1-7a072034f5fe/resource/a0d6e945-eafc-439a-913e-00aa1db90b9e/download/monthly_tourism_indicators.xlsx'
# Load all sheets from the Excel file into a dictionary
sheets = pd.read_excel(url, sheet_name=None, header=None)
# Preview sheet names
print(sheets.keys())

In [ ]:
# Replace spaces, commas, and ampersands in sheet names with underscores
sheets = {
    key.replace('&','and').replace(' ','_').strip(): df
    for key, df in sheets.items()
}

In [ ]:
#Assign each sheet to a nicely named variable
df_travel = sheets['Traveller_Entries_via_BC']
df_food = sheets['Food_Services_and_Drinking_Places']
df_transport = sheets['Transportation_Indicators']
df_other = sheets['Other_Tourism_Sector_Indicators']

In [ ]:
#----- Data Cleaning for Traveller_Entries_via_BC----

In [ ]:
df_travel.head(35) #inspect the first sheet to understand headers and beginning of montly data

In [ ]:
#slices the DataFrame starting at row 31 (beginnign of monthly data) and keeps everything from there onward)
df_travel_monthly = df_travel.iloc[31:].reset_index(drop=True)

In [ ]:
#Building column headers by grabbing the original header rows BEFORE slicing and using the custom function
header_travel = df_travel.iloc[2:4]
df_travel_monthly.columns = build_headers(header_travel)
#check column types and null counts
df_travel_monthly.info()

In [ ]:
#checks every column to see if all rows are NaN so we can confirm before dropping.
df_travel_monthly.columns[df_travel_monthly.isna().all()]

In [ ]:
#Remove empty columns from df_travel_monthly
df_travel_monthly = df_travel_monthly.dropna(axis=1, how='all')

In [ ]:
#check the column names to define duplicates and inspect and see which columns are redundant 
df_travel_monthly.info()

In [ ]:
#Remove unnecessary columns as 1 is sum of 2 and 5, 9 is sum of 10 and 13 we can drop 1 and 9
df_travel_monthly.drop(df_travel_monthly.columns[[1, 9]], axis=1, inplace=True)

In [ ]:
#Change column names to make them distinct
df_travel_monthly.columns = [
    'period',
    'usa_Total',
    'same_day',
    'overnight',
    'overseas_total',
    'asia',
    'europe',
    'other',
    'usa_total_%',
    'same_day_%',
    'overnight_%',
    'overseas_total_%',
    'asia_%',
    'europe_%',
    'other_%'
]

In [ ]:
#drop empty columns
df_travel_monthly = df_travel_monthly.dropna(how='all')

In [ ]:
df_travel_monthly.info()

In [ ]:
# Filter the DataFrame to find rows with missing values in either 'Same_day_%' or 'Overnight_%' columns
df_travel_monthly[df_travel_monthly['same_day_%'].isna() | df_travel_monthly['overnight_%'].isna()]

In [ ]:
#convert certain columns in the DataFrame to numeric data types 
numeric_cols = df_travel_monthly.columns[1:]
df_travel_monthly[numeric_cols] = df_travel_monthly[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
df_travel_monthly.info()

In [ ]:
# Create a list to store clean date labels
clean_periods = []

# Define the first year from the starting 'Jan' entry
first_year = 2000

# This will be incremented as we loop through
year_tracker = first_year

# Loop through each value in the 'Period' column
for entry in df_travel_monthly['period']:
    
    # If 'Jan' contains a year suffix (e.g. 'Jan '01')
    if 'Jan' in entry and "'" in entry:
        try:
            # Extract 2-digit year, convert to full year
            year_suffix = entry.split("'")[-1]
            year_tracker = first_year + int(year_suffix)  # '01' → 2001
        except ValueError:
            pass  # If anything fails, keep the previous year
        month = "Jan"
    
    # If it's just 'Jan' with no year (the last year has it) this will simply increment the year
    elif entry.strip() == "Jan":
        year_tracker += 1  # Manually increment year
        month = "Jan"
    
    else:
        month = entry  # Could be Feb–Dec or any non-January month

    # Combine into clean string like "Feb 2001"
    clean_periods.append(f"{month} {year_tracker}")

# Add new column to DataFrame
df_travel_monthly['period_clean'] = clean_periods


In [ ]:
#Move 'Period_clean' data into the 'Period' column
df_travel_monthly['period'] = df_travel_monthly['period_clean']
df_travel_monthly.drop(columns=['period_clean'], inplace=True)

In [ ]:
#Make 'Period' into true datetime objects:
df_travel_monthly['period'] = pd.to_datetime(df_travel_monthly['period'], format='%b %Y')

In [ ]:
df_travel_monthly.set_index('period', inplace=True)

In [ ]:
df_travel_monthly.tail(20)

In [ ]:
#check if Data Frame is sorted by time
df_travel_monthly.index.is_monotonic_increasing

In [ ]:
df_travel_monthly.head()

In [ ]:
#----- Data Cleaning for Food_Services_and_Drinking_Places----

In [ ]:
df_food.head(35) #inspect the first sheet to understand headers and beginning of montly data

In [ ]:
#slices the DataFrame starting at row 31 (beginnign of monthly data) and keeps everything from there onward)
df_food_monthly = df_food.iloc[32:].reset_index(drop=True)
df_food_monthly.head()

In [ ]:
#Building column headers by grabbing the original header rows BEFORE slicing and using the custom function
header_food = df_food.iloc[2:5]
df_food_monthly.columns = build_headers(header_food)
#check column types and null counts
df_food_monthly.info()

In [ ]:
#checks every column to see if any columns are NaN so we can confirm before dropping.
df_food_monthly.columns[df_food_monthly.isna().all()]

In [ ]:
#Remove empty columns from df_food_monthly
df_food_monthly = df_food_monthly.dropna(axis=1, how='all')

In [ ]:
#check the column names to define duplicates and inspect and see which columns are redundant 
df_food_monthly.info()

In [ ]:
#Remove unnecessary columns 
df_food_monthly.drop(df_food_monthly.columns[[1, 4,7,10]], axis=1, inplace=True)
df_food_monthly.info()

In [ ]:
#Change column names to make them distinct
df_food_monthly.columns = [
    'period',
    'bc_drinking_places',
    'bc_food_services',
    'ca_drinking_places',
    'ca_food_services',
    'bc_drinking_places_%',
    'bc_food_services_%',
    'ca_drinking_places_%',
    'ca_food_services_%',
]

In [ ]:
#drop empty columns
df_food_monthly.dropna(axis=1, how='all', inplace=True)

In [ ]:
#convert certain columns in the DataFrame to numeric data types 
numeric_cols = df_food_monthly.columns[1:]
df_food_monthly[numeric_cols] = df_food_monthly[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
df_food_monthly.info()

In [ ]:
#convert certain columns in the DataFrame to numeric data types 
numeric_cols = df_food_monthly.columns[1:]
df_food_monthly[numeric_cols] = df_food_monthly[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
#remove period columns and all NaN values
df_food_monthly.drop(columns=['period'], inplace=True)
df_food_monthly.dropna(how='all', inplace=True)

In [ ]:
df_food_monthly.tail()

In [ ]:
#----- Data Cleaning for Transportation_Indicators----

In [ ]:
df_transport.head(35) #inspect the first sheet to understand headers and beginning of montly data

In [ ]:
# slices the DataFrame starting at row 31 (beginnign of monthly data) and keeps everything from there onward)
df_transport_monthly = df_transport.iloc[32:].reset_index(drop=True)
df_transport_monthly.head()

In [ ]:
#Building column headers by grabbing the original header rows BEFORE slicing and using the custom function
header_transport = df_transport.iloc[2:5]
df_transport_monthly.columns = build_headers(header_transport)
#check column types and null counts
df_food_monthly.info()

In [ ]:
#checks every column to see if all rows are NaN so we can confirm before dropping.
df_transport_monthly.columns[df_transport_monthly.isna().all()]

In [ ]:
#Remove empty columns from df_transport_monthly
df_transport_monthly = df_transport_monthly.dropna(axis=1, how='all')

In [ ]:
#check the column names to define duplicates and inspect and see which columns are redundant 
df_transport_monthly.info()

In [ ]:
#Drop the columns that has no value in 
df_transport_monthly.drop(columns=[''], inplace=True)
df_transport_monthly.info()

In [ ]:
#Remove unnecessary columns 
df_transport_monthly.drop(
    columns=[
        'Air Passenger Traffic 1,2 Vancouver Total',
        'Air Passenger Traffic Vancouver Total'
    ],
    inplace=True
)
df_transport_monthly.info()

In [ ]:
#Change column names to make them distinct
df_transport_monthly.columns = [
    'period',
    'air_vancouver_domestic',
    'air_vancouver_trans_border',
    'air_vancouver_other_int',
    'victoria_total',
    'bc_ferries_vehicles',
    'bc_ferries_passengers',
    'air_vancouver_domestic_%',
    'air_vancouver_trans_border_%',
    'air_vancouver_other_int_%',
    'victoria_total_%',
    'bc_ferries_vehicles_%',
    'bc_ferries_passengers_%',
]

In [ ]:
#drop empty columns
df_transport_monthly.dropna(axis=1, how='all', inplace=True)

In [ ]:
#convert certain columns in the DataFrame to numeric data types 
numeric_cols = df_transport_monthly.columns[1:]
df_transport_monthly[numeric_cols] = df_transport_monthly[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
df_transport_monthly.info()

In [ ]:
df_transport_monthly.tail(20)

In [ ]:
#Drop the unnecessary last columns and remove the period column
df_transport_monthly = df_transport_monthly.iloc[:304]
df_transport_monthly.drop(columns=['period'], inplace=True)

In [ ]:
#----- Data Cleaning for Other_Tourism_Sector_Indicators----

In [ ]:
df_other.head(35) #inspect the first sheet to understand headers and beginning of montly data

In [ ]:
# slices the DataFrame starting at row 31 (beginnign of monthly data) and keeps everything from there onward)
df_other_monthly = df_other.iloc[31:].reset_index(drop=True)
df_other_monthly.head()

In [ ]:
#Building column headers by grabbing the original header rows BEFORE slicing and using the custom function
header_other = df_other.iloc[2:4]
df_other_monthly.columns = build_headers(header_other)

#check column types and null counts
df_other_monthly.info()

In [ ]:
#checks every column to see if all rows are NaN so we can confirm before dropping.
df_other_monthly.columns[df_other_monthly.isna().all()]

In [ ]:
#Remove empty columns from df_other_monthly
df_other_monthly = df_other_monthly.dropna(axis=1, how='all')

In [ ]:
df_other_monthly.info()

In [ ]:
#Change column names to make them distinct
df_other_monthly.columns = [
    'period',
    'employement_air_transport',
    'employement_accomodation',
    'employement_food_and_beverage',
    'employement_art_entertainment_recreation',
    'hotel_occupancy_rate',
    'hotel_room_rate',
    'hotel_room_revenue',
    'cpi_traveller_accomodation',
    'cpi_restaurant_meals',
    'employement_air_transport_%',
    'employement_accomodation_%',
    'employement_food_and_beverage_%',
    'employement_art_entertainment_recreation_%',
    'hotel_occupancy_rate_%',
    'hotel_room_rate_%',
    'hotel_room_revenue_%',
    'cpi_traveller_accomodation_%',
    'cpi_restaurant_meals_%', 
]

In [ ]:
#convert certain columns in the DataFrame to numeric data types 
numeric_cols = df_other_monthly.columns[1:]
df_other_monthly[numeric_cols] = df_other_monthly[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
df_other_monthly.info()

In [ ]:
df_other_monthly.head(20)

In [ ]:
#Drop the unnecessary last columns and remove the period column
df_other_monthly = df_other_monthly.iloc[:304]
df_other_monthly.drop(columns=['period'], inplace=True)

In [ ]:
df_other_monthly.info()

In [ ]:
###################################################################

In [ ]:
# Share index from df_travel_monthly across all others
df_food_monthly.index        = df_travel_monthly.index
df_transport_monthly.index   = df_travel_monthly.index
df_other_monthly.index       = df_travel_monthly.index


In [ ]:
monthly_tourism_indicators = pd.concat(
    [df_travel_monthly, df_food_monthly, df_transport_monthly, df_other_monthly],
    axis=1
)

In [ ]:
#see how many NaNs are in the data. 
missing_counts = monthly_tourism_indicators.isna().sum().sort_values(ascending=False)
print(missing_counts)

In [ ]:
#If a specific month hotel revenue is missing, fill it using the average of that month's revenue from other years that do have data.
#add a numeric column to help us match months later.
monthly_tourism_indicators['month_num'] = monthly_tourism_indicators.index.month

# Identify all columns with at least one NaN
columns_with_nans = monthly_tourism_indicators.columns[
    monthly_tourism_indicators.isna().any()
]

#Apply seasonal_fill to each of those columns
for col in columns_with_nans:
    filled_col = col + '_filled'
    monthly_tourism_indicators[filled_col] = seasonal_fill(
        monthly_tourism_indicators[col],
        monthly_tourism_indicators['month_num']
    )


In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns when printing remove none for default setting
monthly_tourism_indicators.tail()

In [ ]:
#re orginize the columns to have non fılled (original) and filled column to be side by side
# Step 1: Create a clean interleaved column list
reordered_cols = []

for col in monthly_tourism_indicators.columns:
    if col.endswith('_filled'):
        continue  # Skip _filled for now — we'll add it after its original
    reordered_cols.append(col)
    filled_col = col + '_filled'
    if filled_col in monthly_tourism_indicators.columns:
        reordered_cols.append(filled_col)

# Step 2: Reorder the DataFrame
monthly_tourism_indicators = monthly_tourism_indicators[reordered_cols]


In [ ]:
monthly_tourism_indicators.head()


In [ ]:
# remove # inte cell below to export the finished clean dataframe as csv
#monthly_tourism_indicators.to_csv('monthly_tourism_cleaned.csv', index=True)